In [6]:
import os
import numpy as np
import pandas as pd
import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go
import ipywidgets as wg
from scipy import stats
from IPython.display import display
from urllib.request import urlopen
from plotly.subplots import make_subplots

psldata = pd.ExcelFile(r'/Users/Amar/Documents/WD/projects/PSLVisualisation/data/All deliveries.xlsx')
df = pd.read_excel(psldata)

df.iloc[:,-2:] = df.iloc[:,-2:].fillna('No dismisal')

print(df)



      Match_id  Innings  Over  Ball       Batting_team      Bowling_team  \
0            1        1     0     1     Peshawar Zalmi  Islamabad United   
1            1        1     0     2     Peshawar Zalmi  Islamabad United   
2            1        1     0     3     Peshawar Zalmi  Islamabad United   
3            1        1     0     4     Peshawar Zalmi  Islamabad United   
4            1        1     0     5     Peshawar Zalmi  Islamabad United   
...        ...      ...   ...   ...                ...               ...   
5409        24        2    15     6  Quetta Gladiators    Peshawar Zalmi   
5410        24        2    15     7  Quetta Gladiators    Peshawar Zalmi   
5411        24        2    16     1  Quetta Gladiators    Peshawar Zalmi   
5412        24        2    16     2  Quetta Gladiators    Peshawar Zalmi   
5413        24        2    16     3  Quetta Gladiators    Peshawar Zalmi   

       Batsman_strike     Non_striker          Bowler  Batsman_runs  \
0     Mohammad H

In [11]:

##importing libraries

import os
import numpy as np
import pandas as pd
import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go
import ipywidgets as wg
from scipy import stats
from IPython.display import display
from urllib.request import urlopen
from plotly.subplots import make_subplots


#importing data file
psldata = pd.ExcelFile(r'/Users/Amar/Documents/WD/projects/PSLVisualisation/data/All deliveries.xlsx')
df = pd.read_excel(psldata)


##cleaning data
#Replaced Nas
df.iloc[:,-2:] = df.iloc[:,-2:].fillna('No dismisal')


# In[324]:


def wick(x):
    if x != "No dismisal":
        return 1
    else:
        return 0
df['wicket'] = df['Dismisal_kind'].apply(lambda x: wick(x))




def isSix(x):
    if x == 6:
        return 1
    else:
        return 0
    
def isFour(x):
    if x == 4:
        return 1
    else:
        return 0
df['Six'] = df['Batsman_runs'].map(lambda x: isSix(x))
df['Four'] = df['Batsman_runs'].map(lambda x: isFour(x))



#Ball-by-ball detail for each batsman

df_bat = pd.merge(df.groupby(['Batsman_strike','Match_id']).sum().reset_index()[['Batsman_strike','Match_id','Batsman_runs']], df.groupby(['Batsman_strike', 'Match_id']).count().reset_index()[['Batsman_strike','Match_id', 'Ball']].rename(columns = {'Ball': 'Balls_played'}), on = ['Batsman_strike', 'Match_id'])



##Batting datafrane
df_batAvg = pd.merge(pd.merge(df_bat.groupby('Batsman_strike').mean().reset_index()[['Batsman_strike','Batsman_runs', 'Balls_played']].rename(columns = {'Batsman_runs' : 'AvgRuns_match' , 'Balls_played': 'AvgBalls_match'}), df[['Batsman_strike', 'Batting_team']].drop_duplicates(keep = 'first'), on = 'Batsman_strike') , pd.merge(df.groupby(['Batsman_strike']).sum().reset_index()[['Batsman_strike', 'Batsman_runs', 'Six', 'Four']].rename(columns = {'Batsman_runs': 'TotalRuns', 'Six':'TotalSixes', 'Four':'TotalFours'}), df.groupby(['Batsman_strike']).count().reset_index()[['Batsman_strike' , 'Ball']].rename(columns = {'Ball': 'Balls_played'}), on ='Batsman_strike'), on = 'Batsman_strike')
                                                                                                                                     
                                                                                                                                     

df_batAvg['Runs_sq'] = 0
df_batAvg['Matches_played'] = 0


#To calculate variance
for i in range(len(df_batAvg)):
    for j in range(len(df_bat)):
        if df_batAvg.iloc[i]['Batsman_strike'] == df_bat.iloc[j]['Batsman_strike']:
            df_batAvg['Runs_sq'].iloc[i] += df_bat.iloc[j]['Batsman_runs'] **2
            df_batAvg['Matches_played'].iloc[i] += 1


#Strke rate, % boundary and Variance
df_batAvg['Strike_rate'] = df_batAvg['TotalRuns'] * 100 /df_batAvg['Balls_played']
df_batAvg['PercentageRuns_boundary'] = (df_batAvg['TotalSixes']* 6 + df_batAvg['TotalFours'] * 4) * 100/df_batAvg['TotalRuns']
df_batAvg["Var"] = 
(df_batAvg['Runs_sq'] - df_batAvg['Matches_played']*(df_batAvg['AvgRuns_match']**2))/(df_batAvg['Matches_played'] - 1)
df_batAvg = df_batAvg.sort_values(by = 'TotalRuns', ascending = False)


##Batsman Scatter

z = df_batAvg['Batsman_strike']
trace = go.Scatter(y = df_batAvg['Strike_rate'], x= df_batAvg['AvgRuns_match'], mode = 'markers', showlegend = False , text = df_batAvg['Batsman_strike'] + "  " + df_batAvg['Batting_team'], hoverinfo =  "text", marker = dict(size = 20, symbol = 'circle' , cmin = 0, autocolorscale = True, reversescale = True, colorbar = {'title' : {'text': 'Score variance'}}, color = df_batAvg['Var']))
data = [trace]

layout = go.Layout(paper_bgcolor = 'white',plot_bgcolor = 'white' , yaxis = dict(title = dict(text = "Strike rate" , font = dict(color ='black')) ,showgrid = True, zeroline = False, color = 'black'), xaxis = dict( title= dict( text = 'Average runs'), showgrid = False, zeroline= False), title = dict(text = "All batsman's performance - Average runs, Strike rate, Score variance"))
fig = go.Figure(data = data, layout = layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.plot(fig, include_plotlyjs = False, output_type = 'div')

plotly.plot(fig)


#Batsman Histogram


# team_list = list(df_batAvg['Batting_team'].unique())

# trace0 = go.Histogram(x= df_batAvg.loc[df_batAvg['Batting_team'] == team_list[0], 'AvgRuns_match'], opacity = 0.75, name = team_list[0])
# trace1 = go.Histogram(x= df_batAvg.loc[df_batAvg['Batting_team'] == team_list[1], 'AvgRuns_match'], opacity = 0.75, name = team_list[1])
# trace2 = go.Histogram(x= df_batAvg.loc[df_batAvg['Batting_team'] == team_list[2], 'AvgRuns_match'], opacity = 0.75, name = team_list[2])
# trace3 = go.Histogram(x= df_batAvg.loc[df_batAvg['Batting_team'] == team_list[3], 'AvgRuns_match'], opacity = 0.75, name = team_list[3])
# trace4 = go.Histogram(x= df_batAvg.loc[df_batAvg['Batting_team'] == team_list[4], 'AvgRuns_match'], opacity = 0.75, name = team_list[4])

# fig4 = make_subplots(rows =3, cols = 2)
# fig4.append_trace(trace0,1,1)
# fig4.append_trace(trace1,2,1)
# fig4.append_trace(trace2,2,2)
# fig4.append_trace(trace3,3,1)
# fig4.append_trace(trace4,3,2)

# fig4.layout.title = dict(text = 'Distribution of average batsman scores')
# fig4.layout.xaxis = dict(tickmode = 'linear', dtick = 10)


# plotly.offline.plot(fig4, include_plotlyjs = False, output_type = 'div')



def runlevel(x):
    if x >= 1.25*df_batAvg['AvgRuns_match'].mean():
        return 'High'
    elif x <= 0.75 * df_batAvg['AvgRuns_match'].mean():
        return 'Low'
    else:
        return 'Average'
    
def strikelevel(x):
    if x >= 1.25*df_batAvg['Strike_rate'].mean():
        return 'High'
    elif x <= 0.75 * df_batAvg['Strike_rate'].mean():
        return 'Low'
    else:
        return 'Average'
    
def varlevel(x):
    if x <= 0.75*df_batAvg['Var'].mean():
        return 'High Consistency'
    elif x >= 1.25 * df_batAvg['Var'].mean():
        return 'Inconsistent'
    else:
        return 'Average'
    

df_batAvg['Runs_level'] = df_batAvg['AvgRuns_match'].map(lambda x: runlevel(x))
df_batAvg['Strike_level'] = df_batAvg['Strike_rate'].map(lambda x: strikelevel(x))
df_batAvg['Var_level'] = df_batAvg['Var'].map(lambda x: varlevel(x))


dfteam = df.groupby(['Batting_team','Bowling_team', 'Match_id', 'Innings', 'Over']).sum().reset_index()
over = 0
total = 0
Total_runs = []

for i in range(len(dfteam)):
    if dfteam.iloc[i]['Over'] >= over:
        total =+ total + dfteam.iloc[i]['Batsman_runs'] + dfteam.iloc[i]['Extra_runs']
        over =+ 1
        Total_runs.append(total)
    else:
        total = dfteam.iloc[i]['Batsman_runs'] + dfteam.iloc[i]['Extra_runs']
        Total_runs.append(total)
        over = 1


dfteam['Total_runs'] = pd.Series(Total_runs)



# ## Defining bowler metrics
# def caught(x):
#     if x == 'caught':
#         return 1
#     else:
#         return 0
# def bowled(x):
#     if x == bowled:
#         return 1
#     else:
#         return 0
# def runout(x):
#     if x == 'runout':
#         return 1
#     else:
#         return 0
# def lbw(x):
#     if x == 'lbw':
#         return 1
#     else:
#         return 0
# def stump(x):
#     if x == 'stump':
#         return 1
#     else:
#         return 0
# def candb(x):
#     if x == 'caught and bowled':
#         return 1
#     else:
#         return 0


# df['Caught'] = df['Dismisal_kind'].apply(lambda x: caught(x))
# df['Bowled'] = df['Dismisal_kind'].apply(lambda x: bowled(x))
# df['Lbw'] = df['Dismisal_kind'].apply(lambda x: lbw(x))
# df['Runout'] = df['Dismisal_kind'].apply(lambda x: runout(x))
# df['Stump'] = df['Dismisal_kind'].apply(lambda x: stump(x))
# df['Caught_bowled'] = df['Dismisal_kind'].apply(lambda x: candb(x))


# ## Creating a bowler dataframe
# df_bowl = pd.merge(df.groupby(['Bowler', 'Match_id']).sum().reset_index().filter(regex = "[Innings,Over,Ball]"), df.groupby(['Bowler', 'Match_id']).count().reset_index()[['Bowler','Match_id', 'Ball']].rename(columns = {'Ball': 'TotalDeliveries'}), on = ['Bowler', 'Match_id'])

# df_bowlAvg = pd.merge(df.groupby(['Bowler', 'Match_id']).sum().reset_index().filter(regex = "[^Match_id,Innings,Over,Ball]") ,  df.groupby(['Bowler', 'Match_id']).count().reset_index()[['Bowler', 'Ball']].rename(columns = {'Ball': 'Delieveries_bowled'}), on = 'Bowler').groupby('Bowler').mean().reset_index()


# df_bowl_Avg_merge = pd.merge(df_bowl.groupby('Bowler').sum().reset_index().filter(regex = '[^Match_id,Innings,Over,Ball]'), df_bowlAvg.filter(regex = "[^Caught,Bowled,Stump,Caught_bowled, Lbw, Runout]").rename(columns = {'Delieveries_bowled':'AvgDeliveries_bowled','Batsman_runs': 'AvgBatsman_runs', 'Extra_runs': 'AvgExtra_runs' , 'Six': 'AvgSix', 'Four': 'AvgFour', 'wicket' : 'AvgWicket'}),
#  on = 'Bowler')

# dfecon = df.groupby(['Bowler','Match_id', 'Over']).sum().reset_index().groupby('Bowler').mean().reset_index()[['Bowler','Batsman_runs', 'Extra_runs']]
# dfecon['Economy'] = dfecon['Batsman_runs'] + dfecon['Extra_runs']



# df_bowler = pd.merge(df_bowl_Avg_merge, dfecon[['Bowler','Economy']], on = 'Bowler')


# ## Bowler variance
# df_bowler['Runs_sq'] = 0
# df_bowler['Matches_played'] = 0
# for i in range(len(df_bowler)):
#     for j in range(len(df_bowl)):
#         if df_bowler.iloc[i]['Bowler'] == df_bowl.iloc[j]['Bowler']:
#             df_bowler['Runs_sq'].iloc[i] += (df_bowl.iloc[j]['Batsman_runs'] +  df_bowl.iloc[j]['Extra_runs'])**2
#             df_bowler['Matches_played'].iloc[i] += 1


# df_bowler['Var'] = (df_bowler['Runs_sq'] - (df_bowler['Matches_played']*((df_bowler['AvgBatsman_runs']+df_bowler['AvgExtra_runs'])**2)))/(df_bowler['Matches_played'] -1)

 

# df_bowler = pd.merge(df_bowler, df[['Bowler', 'Bowling_team']].drop_duplicates(keep = 'first'), on = 'Bowler')
# df_bowler.dropna(inplace = True)


# ##Bowler Scatter chart

# trace1 = go.Scatter(x = df_bowler['wicket'], y = df_bowler['Economy'], mode = 'markers', hoverinfo = 'text' , text = df_bowler['Bowler'] + "   " + df_bowler['Bowling_team'] , marker = dict(size = 25, symbol = 'circle' , cmin = 0, autocolorscale = True, reversescale = True, colorbar = {'title' : {'text': 'Bowler Variance'}}, color = df_bowler['Var']))

# layout = go.Layout(paper_bgcolor = 'white',plot_bgcolor = 'white' , hovermode = 'closest',  yaxis = dict(title = dict(text = "Economy" , font = dict(color ='black')) ,showgrid = True, zeroline = False, color = 'black'), xaxis = dict(title = dict( text = 'Total wickets'), showgrid = False, zeroline = False), title = dict(text = "All Bowler's Performance - Economy, Total Wickets, Variance"))


# data = [trace1]

# fig22 = go.Figure(data = data, layout = layout)

# plotly.offline.init_notebook_mode(connected=True)
# plotly.offline.plot(fig22, include_plotlyjs = False, output_type = 'div')


# ##Bowler histogram


# team_list = ['Peshawar Zalmi', 'Karachi Kings', 'Islamabad United', 'Quetta Gladiators', 'Lahore Qalandars']

# trace0 = go.Histogram(x= df_bowler.loc[df_bowler['Bowling_team'] == team_list[0], 'Economy'], opacity = 0.75, name = team_list[0])
# trace1 = go.Histogram(x= df_bowler.loc[df_bowler['Bowling_team'] == team_list[1], 'Economy'], opacity = 0.75, name = team_list[1])
# trace2 = go.Histogram(x= df_bowler.loc[df_bowler['Bowling_team'] == team_list[2], 'Economy'], opacity = 0.75, name = team_list[2])
# trace3 = go.Histogram(x= df_bowler.loc[df_bowler['Bowling_team'] == team_list[3], 'Economy'], opacity = 0.75, name = team_list[3])
# trace4 = go.Histogram(x= df_bowler.loc[df_bowler['Bowling_team'] == team_list[4], 'Economy'], opacity = 0.75, name = team_list[4])

# fig4 = plotly.tools.make_subplots(rows = 3, cols = 2)
# fig4.append_trace(trace0,1,1)
# fig4.append_trace(trace1,1,2)
# fig4.append_trace(trace2,2,1)
# fig4.append_trace(trace3,2,2)
# fig4.append_trace(trace4,3,1)

# fig4.layout.title = dict(text = "Distribution of average bowler economy for each team")

# plotly.offline.plot(fig4, include_plotlyjs = False, output_type = 'div')


# ##Setting up a ranking datafrane

# dfscore = df_batAvg[['Batsman_strike', 'Batting_team','AvgRuns_match', 'Strike_rate', 'Var']].rename(columns = {'Batsman_strike': 'Player', 'Batting_team':'Team','Var': 'Bat_var'}).merge(df_bowler[['Bowler', 'Economy', 'AvgWicket','AvgDeliveries_bowled', 'Var', 'Bowling_team']].rename(columns = {'Bowler':'Player','Var':'Bowl_var','Bowling_team':'Team'}), on = ['Player','Team'], how = 'outer').fillna(value = 0)                                                                                                          


# ##Player Ranking Metrics

# f = 'Strike_rate'
# g = 'AvgRuns_match'
# h = 'AvgWicket'


# Batsman = np.quantile(dfscore['AvgRuns_match'], 0.5)
# Batvarmean = dfscore.loc[dfscore['AvgRuns_match'] >= Batsman, "Bat_var"].mean()


# Bowlvarmean = dfscore.loc[dfscore['AvgDeliveries_bowled'] >= 0, "Bowl_var"].mean()

# dfscore['BatVarRank'] = np.zeros(len(dfscore))

# dfscore['BowlVarRank'] = np.zeros(len(dfscore))


# def greaterf(x):
#         if x == 0:
#             return 0
#         if x >= 1.25 * dfscore[f].mean():
#             return 3
#         if x<= 0.75 *dfscore[f].mean():
#             return  1
#         else:
#             return 2

# def greaterg(x):
#         if x == 0:
#             return 0
#         if x >= 1.25 * dfscore[g].mean():
#             return 3
#         if x<= 0.75 *dfscore[g].mean():
#             return  1
#         else:
#             return 2

# def greaterh(x):
#         if x == 0:
#             return 0
#         if x >= 1.25 * dfscore[h].mean():
#             return 3
#         if x<= 0.75 *dfscore[h].mean():
#             return  1
#         else:
#             return 2


# dfscore[f + str('Rank')] = dfscore[f].map(lambda x: greaterf(x))    
# dfscore[g + str('Rank')] = dfscore[g].map(lambda x: greaterg(x))
# dfscore[h + str('Rank')] = dfscore[h].map(lambda x: greaterh(x))

# def ecorank(x):
#     if x == 0:
#         return 0
#     elif x >= 1.25 * dfscore.loc[dfscore['AvgDeliveries_bowled'] >= 0, "Economy"].mean():
#         return 1
#     elif x <= 0.75 * dfscore.loc[dfscore['AvgDeliveries_bowled'] >= 0, "Economy"].mean():
#         return 3
#     else:
#         return 2
    
# dfscore['EconomyRank'] = dfscore['Economy'].map(lambda x: ecorank(x))


# for i in range(len(dfscore)):
#     if dfscore.iloc[i]['AvgRuns_match'] <= Batsman:
#         dfscore['BatVarRank'].iloc[i] = 0
#     elif dfscore['Bat_var'].iloc[i] >= (1.25 * Batvarmean):  
#         dfscore['BatVarRank'].iloc[i] = 1
#     elif dfscore.iloc[i]['Bat_var'] <= 0.75 * Bowlvarmean:
#         dfscore['BatVarRank'].iloc[i] = 3
#     else:
#         dfscore['BatVarRank'].iloc[i] = 2
        
        
        
# for i in range(len(dfscore)):
#     if dfscore.iloc[i]['AvgDeliveries_bowled'] == 0:
#         dfscore['BowlVarRank'].iloc[i] = 0
#     elif dfscore['Bowl_var'].iloc[i] >= (1.25 * Bowlvarmean):  
#         dfscore['BowlVarRank'].iloc[i] = 1
#     elif dfscore.iloc[i]['Bowl_var'] <= 0.75 * Bowlvarmean:
#         dfscore['BowlVarRank'].iloc[i] = 3
#     else:
#         dfscore['BowlVarRank'].iloc[i] = 2



# ##Creating PlayerScore column as the sum of all metrics

# dfscore['PlayerScore'] = dfscore['AvgRuns_matchRank'] + dfscore['Strike_rateRank'] + dfscore['AvgWicketRank'] + dfscore['BatVarRank'] + dfscore['BowlVarRank'] + dfscore['EconomyRank']


# ##Ranking levels

# def mvprank(x):
#     if x >= np.quantile(dfscore['PlayerScore'], 0.90):
#         return 4
#     elif x >= np.quantile(dfscore['PlayerScore'], 0.75):
#             return 3
#     elif x <= np.quantile(dfscore['PlayerScore'], 0.50):
#             return 1
#     else:
#         return 2
    
# dfscore['PlayerRank'] = dfscore['PlayerScore'].map(lambda x: mvprank(x))
# dfscore['MVP'] = np.zeros(len(dfscore))
# dfscore.loc[dfscore['PlayerRank']==4,'MVP'] = 'MVP'
# dfscore.loc[dfscore['PlayerRank']==3, 'MVP'] = 'VP'
# dfscore.loc[dfscore['PlayerRank'] <3, 'MVP'] = 'Below Average'


# ## Top 10 Players
# dfscore.sort_values(by = 'PlayerScore', ascending = False).head(10)


# #Boxplot

# trace0 = go.Box(y = dfscore.loc[dfscore['Team'] == 'Peshawar Zalmi', 'PlayerScore'], name = 'Peshawar Zalmi', boxmean = 'sd', boxpoints = 'all')
# trace1 = go.Box(y = dfscore.loc[dfscore['Team'] == 'Islamabad United', 'PlayerScore'], name = 'Islamabad United', boxmean = 'sd', boxpoints = 'all')
# trace2 = go.Box(y = dfscore.loc[dfscore['Team'] == 'Quetta Gladiators', 'PlayerScore'], name = 'Quetta Gladiators', boxmean = 'sd', boxpoints = 'all')
# trace3 = go.Box(y = dfscore.loc[dfscore['Team'] == 'Karachi Kings', 'PlayerScore'], name = 'Karachi Kings' , boxmean = 'sd', boxpoints = 'all')
# trace4 = go.Box(y = dfscore.loc[dfscore['Team'] == 'Lahore Qalandars', 'PlayerScore'], name = 'Lahore Qalandars' , boxmean = 'sd', boxpoints = 'all')

# data = [trace0, trace1, trace2, trace3, trace4]

# fig111 = go.Figure(data = data)
# fig111.layout.title = dict(text = "Player Score distribution for each team" )

# import bs4
# from bs4 import BeautifulSoup

# plotly.offline.plot(fig111, include_plotlyjs = False, output_type = 'div')


TypeError: plot() missing 1 required positional argument: 'kind'

In [283]:
df_batAvg

,Batsman_strike,AvgRuns_match,AvgBalls_match,Batting_team,TotalRuns,TotalSixes,TotalFours,Balls_played,Runs_sq,Matches_played,Strike_rate,PercentageRuns_boundary,Var
35,Kamran Akmal,32.090909,24.909091,Peshawar Zalmi,353,16,32,274,24767,11,128.832117,63.456091,1343.890909
10,Babar Azam,29.100000,26.500000,Karachi Kings,291,4,36,265,11959,10,109.811321,57.731959,387.877778
18,DR Smith,30.444444,27.444444,Islamabad United,274,10,29,247,12976,9,110.931174,64.233577,579.277778
52,RR Rossouw,31.875000,26.250000,Quetta Gladiators,255,9,18,210,12553,8,121.428571,49.411765,632.125000
34,KP Pietersen,34.428571,23.142857,Quetta Gladiators,241,14,15,162,15795,7,148.765432,59.751037,1249.619048
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,MN van Wyk,1.000000,7.000000,Quetta Gladiators,1,0,0,7,1,1,14.285714,0.000000,NaN
79,Zulfiqar Babar,0.500000,1.500000,Quetta Gladiators,1,0,0,3,1,2,33.333333,0.000000,0.500000
76,Usman Khan,0.000000,2.000000,Karachi Kings,0,0,0,2,0,1,0.000000,NaN,NaN
55,Rumman Raees,0.000000,4.000000,Islamabad United,0,0,0,4,0,1,0.000000,NaN,NaN


In [373]:
import plotly.express as px

def wick(x):
    if x != "No dismisal":
        return 1
    else:
        return 0
df['wicket'] = df['Dismisal_kind'].apply(lambda x: wick(x))


def isSix(x):
    if x == 6:
        return 1
    else:
        return 0
    
def isFour(x):
    if x == 4:
        return 1
    else:
        return 0
df['Six'] = df['Batsman_runs'].map(lambda x: isSix(x))
df['Four'] = df['Batsman_runs'].map(lambda x: isFour(x))



#Ball-by-ball detail for each batsman

df_bat = pd.merge(df.groupby(['Batsman_strike','Match_id']).sum().reset_index()[['Batsman_strike','Match_id','Batsman_runs']], df.groupby(['Batsman_strike', 'Match_id']).count().reset_index()[['Batsman_strike','Match_id', 'Ball']].rename(columns = {'Ball': 'Balls_played'}), on = ['Batsman_strike', 'Match_id'])



##Batting datafrane
df_batAvg = pd.merge(pd.merge(df_bat.groupby('Batsman_strike').mean().reset_index()[['Batsman_strike','Batsman_runs', 'Balls_played']].rename(columns = {'Batsman_runs' : 'AvgRuns_match' , 'Balls_played': 'AvgBalls_match'}), df[['Batsman_strike', 'Batting_team']].drop_duplicates(keep = 'first'), on = 'Batsman_strike') , pd.merge(df.groupby(['Batsman_strike']).sum().reset_index()[['Batsman_strike', 'Batsman_runs', 'Six', 'Four']].rename(columns = {'Batsman_runs': 'TotalRuns', 'Six':'TotalSixes', 'Four':'TotalFours'}), df.groupby(['Batsman_strike']).count().reset_index()[['Batsman_strike' , 'Ball']].rename(columns = {'Ball': 'Balls_played'}), on ='Batsman_strike'), on = 'Batsman_strike')
                                                                                                                                     
                                                                                                                                     

df_batAvg['Runs_sq'] = 0
df_batAvg['Matches_played'] = 0


#To calculate variance
for i in range(len(df_batAvg)):
    for j in range(len(df_bat)):
        if df_batAvg.iloc[i]['Batsman_strike'] == df_bat.iloc[j]['Batsman_strike']:
            df_batAvg['Runs_sq'].iloc[i] += df_bat.iloc[j]['Batsman_runs'] **2
            df_batAvg['Matches_played'].iloc[i] += 1


#Strke rate, % boundary and Variance
df_batAvg['Strike_rate'] = df_batAvg['TotalRuns'] * 100 /df_batAvg['Balls_played']
df_batAvg['PercentageRuns_boundary'] = (df_batAvg['TotalSixes']* 6 + df_batAvg['TotalFours'] * 4) * 100/df_batAvg['TotalRuns']

NumberMatches = df_batAvg['Matches_played']
AvgRuns = df_batAvg['AvgRuns_match']
df_batAvg["Var"] = 0

for i in range(len(df_batAvg)):
    if NumberMatches.loc[i] > 0:
        df_batAvg["Var"].loc[i] = (df_batAvg['Runs_sq'].loc[i] - NumberMatches.loc[i]*(AvgRuns.loc[i]**2))/(NumberMatches.loc[i] - 1)

df_batAvg = df_batAvg.sort_values(by = 'TotalRuns', ascending = False)


##Batsman Scatter

z = df_batAvg['Batsman_strike']
trace = go.Scatter(y = df_batAvg['Strike_rate'], x= df_batAvg['AvgRuns_match'], mode = 'markers', showlegend = False , text = df_batAvg['Batsman_strike'] + "  " + df_batAvg['Batting_team'], hoverinfo =  "text", marker = dict(size = 20, symbol = 'circle' , cmin = 0, autocolorscale = True, reversescale = True, colorbar = {'title' : {'text': 'Score variance'}}, color = df_batAvg['Var']))
data = [trace]

layout = go.Layout(
    paper_bgcolor = 'white',plot_bgcolor = 'white' , yaxis = 
    dict(title = dict(text = "Strike rate" , font = dict(color ='black')) ,showgrid = True, zeroline = False, color = 'black'), 
    xaxis = dict( title= dict( text = 'Average runs'), showgrid = False, zeroline= False), title = dict(text = "All batsman's performance - Average runs, Strike rate, Score variance")
    )
fig = go.Figure(data = data, layout = layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.plot(fig, include_plotlyjs = False, output_type = 'div')

# px.scatter(y = df_batAvg['Strike_rate'], x= df_batAvg['AvgRuns_match'], color=df_batAvg["Var"])
fig.show()

##
##Queetta has 6, pesh, lahore has 4, khi, Isl has 3 in top quadrant
##prints


In [258]:
def quadrant_chart(x, y, team,var=None, xtick_labels=None, ytick_labels=None, highlight_quadrants=None):
    """
    Create the classic four-quadrant chart.
    Args:
        x -- array-like, the x-coordinates to plot
        y -- array-like, the y-coordinates to plot
        xtick_labels -- list, default: None, a two-value list xtick labels
        ytick_labels -- list, default: None, a two-value list of ytick labels
        data_labels -- array-like, default: None, data point annotations
        highlight_quadrants -- list, default: None, list of quadrants to
            emphasize (quadrants are numbered 1-4)
        ax -- matplotlib.axes object, default: None, the user can pass their own
            axes object if desired
    """
    # allow user to specify their own axes
    # ax = ax if ax else plt.axes()

    data = pd.DataFrame({'x': x, 'y': y, 'team':team, 'var':var})

    # calculate averages up front to avoid repeated calculations
    y_avg = data['y'].mean()
    x_avg = data['x'].mean()

    # determine which points to highlight
    # if highlight_quadrants:
    #     quadrants = []
    #     for x_val, y_val in zip(x, y):
    #         q = []
    #         if (x_val >= x_avg) and (y_val >= y_avg):
    #             q.append(1)
    #         if (x_val <= x_avg) and (y_val >= y_avg):
    #             q.append(2)
    #         if (x_val <= x_avg) and (y_val <= y_avg):
    #             q.append(3)
    #         if (x_val >= x_avg) and (y_val <= y_avg):
    #             q.append(4)
    #         quadrants.append(q)
    #     data['quadrant'] = quadrants

    #     # boolean mask - True = highlight, False = don't highlight
    #     highlight = data['quadrant'].apply(lambda q: len(set(
    #     highlight_quadrants) & set(q)) > 0)

    #     # plot the non-highlighted points within the conditional block
    #     fig= px.scatter(data['x'][~highlight], data['y'][~highlight], color=data['bat_team'][~highlight])
    #     data = data[highlight]

    # plot remaining points and quadrant lines
    # fig = px.scatter(x=data['x'], y=data['y'], color= data['var'])
    # fig.add_hline(y= y_avg, line_color='red', line_width=1)
    # fig.add_vline(x = x_avg, line_color='green', line_width=1)
    
    fig1= px.scatter(x=data['x'], y=data['y'], color= data['team'])
    fig1.add_hline(y= y_avg, line_color='red', line_width=1)
    fig1.add_vline(x = x_avg, line_color='green', line_width=1)
    fig.show()
    fig1.show()

    # # add data labels
    # for ix, row in data.iterrows():
    #     px.annotate(row['data_labels'], (row['x'], row['y']), xytext=(2, 5),
    #     textcoords='offset pixels')

In [259]:

quadrant_chart(
    x=df_batAvg['AvgRuns_match'],
    y=df_batAvg['Strike_rate'],
    team=df_batAvg['Batting_team'],
    var=df_batAvg["Var"],
    xtick_labels=['Low', 'High'],
    ytick_labels=['Low', 'High'],
    highlight_quadrants=[1]
)
# plt.title('Strike vs Runs', fontsize=16)
# plt.ylabel('Strike', fontsize=14)
# plt.xlabel('Avg Runs', fontsize=14)

In [242]:
trace0 = go.Box(y = df_batAvg.loc[df_batAvg['Batting_team'] == 'Peshawar Zalmi', 'AvgRuns_match'], name = 'Peshawar Zalmi', boxmean = 'sd', boxpoints = 'all')
trace1 = go.Box(y = df_batAvg.loc[df_batAvg['Batting_team'] == 'Islamabad United', 'AvgRuns_match'], name = 'Islamabad United', boxmean = 'sd', boxpoints = 'all')
trace2 = go.Box(y = df_batAvg.loc[df_batAvg['Batting_team'] == 'Quetta Gladiators', 'AvgRuns_match'], name = 'Quetta Gladiators', boxmean = 'sd', boxpoints = 'all')
trace3 = go.Box(y = df_batAvg.loc[df_batAvg['Batting_team'] == 'Karachi Kings', 'AvgRuns_match'], name = 'Karachi Kings' , boxmean = 'sd', boxpoints = 'all')
trace4 = go.Box(y = df_batAvg.loc[df_batAvg['Batting_team'] == 'Lahore Qalandars', 'AvgRuns_match'], name = 'Lahore Qalandars' , boxmean = 'sd', boxpoints = 'all')

data = [trace0, trace1, trace2, trace3, trace4]

fig111 = go.Figure(data = data)
fig111.layout.title = dict(text = "Batting Avg distribution for each team" )
fig111.show()

##Quetta, Lahore, Pesh similar
##Comparable mean, median

In [262]:
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots

# ##Batsman Histogram


# team_list = list(df_batAvg['Batting_team'].unique())

# trace0 = go.Histogram(x= df_batAvg.loc[df_batAvg['Batting_team'] == team_list[0], 'AvgRuns_match'], opacity = 0.75, name = team_list[0])
# trace1 = go.Histogram(x= df_batAvg.loc[df_batAvg['Batting_team'] == team_list[1], 'AvgRuns_match'], opacity = 0.75, name = team_list[1])
# trace2 = go.Histogram(x= df_batAvg.loc[df_batAvg['Batting_team'] == team_list[2], 'AvgRuns_match'], opacity = 0.75, name = team_list[2])
# trace3 = go.Histogram(x= df_batAvg.loc[df_batAvg['Batting_team'] == team_list[3], 'AvgRuns_match'], opacity = 0.75, name = team_list[3])
# trace4 = go.Histogram(x= df_batAvg.loc[df_batAvg['Batting_team'] == team_list[4], 'AvgRuns_match'], opacity = 0.75, name = team_list[4])

# fig4 = make_subplots(rows =3, cols = 2)
# fig4.append_trace(trace0,1,1)
# fig4.append_trace(trace1,2,1)
# fig4.append_trace(trace2,2,2)
# fig4.append_trace(trace3,3,1)
# fig4.append_trace(trace4,3,2)

# fig4.layout.title = dict(text = 'Distribution of average batsman scores')
# fig4.layout.xaxis = dict(tickmode = 'linear', dtick = 10)

# fig4.show()
# plotly.offline.plot(fig4, include_plotlyjs = False, output_type = 'div')

In [386]:
#Bowling
## Defining bowler metrics
def caught(x):
    if x == 'caught':
        return 1
    else:
        return 0
def bowled(x):
    if x == bowled:
        return 1
    else:
        return 0
def runout(x):
    if x == 'runout':
        return 1
    else:
        return 0
def lbw(x):
    if x == 'lbw':
        return 1
    else:
        return 0
def stump(x):
    if x == 'stump':
        return 1
    else:
        return 0
def candb(x):
    if x == 'caught and bowled':
        return 1
    else:
        return 0


df['Caught'] = df['Dismisal_kind'].apply(lambda x: caught(x))
df['Bowled'] = df['Dismisal_kind'].apply(lambda x: bowled(x))
df['Lbw'] = df['Dismisal_kind'].apply(lambda x: lbw(x))
df['Runout'] = df['Dismisal_kind'].apply(lambda x: runout(x))
df['Stump'] = df['Dismisal_kind'].apply(lambda x: stump(x))
df['Caught_bowled'] = df['Dismisal_kind'].apply(lambda x: candb(x))


## Creating a bowler dataframe
#By each match
df_bowl_match = pd.merge(
    df.groupby(['Bowler', 'Match_id']).sum().reset_index().filter(regex = "[Innings,Over,Ball]"), 
    df.groupby(['Bowler', 'Match_id']).count().reset_index()[['Bowler','Match_id', 'Ball']].rename(columns =        {'Ball': 'TotalDeliveries'}
    ), on = ['Bowler', 'Match_id'])

#Avg deliveries bye ach match
df_bowl_matchAvg = pd.merge(
    df.groupby(['Bowler', 'Match_id']).sum().reset_index().filter(regex = "[^Match_id,Innings,Over,Ball]") ,       df.groupby(['Bowler', 'Match_id']).count().reset_index()[['Bowler', 'Ball']].rename(columns = {'Ball': 'Delieveries_bowled'}), on = 'Bowler').groupby('Bowler').mean().reset_index()

#Avg tot runs by each match
df_bowl_Avg_Tot = pd.merge(
    df_bowl_match.groupby('Bowler').sum().reset_index().filter(regex = '[^Match_id,Innings,Over,Ball]'),           df_bowlAvg.filter(regex = "[^Caught,Bowled,Stump,Caught_bowled, Lbw, Runout]").rename(columns =      {'Delieveries_bowled':'AvgDeliveries_bowled','Batsman_runs': 'AvgBatsman_runs', 'Extra_runs': 'AvgExtra_runs' , 'Six': 'AvgSix', 'Four': 'AvgFour', 'wicket' : 'AvgWicket'}),
 on = 'Bowler')

#Bowler Economy
dfecon = df.groupby(['Bowler','Match_id', 'Over']).sum().reset_index().groupby('Bowler').mean().reset_index()[['Bowler','Batsman_runs', 'Extra_runs']]
dfecon['Economy'] = dfecon['Batsman_runs'] + dfecon['Extra_runs']

#Bowler final dataframe
df_bowler_summ = pd.merge(df_bowl_Avg_Tot.loc[df_bowl_Avg_Tot['TotalDeliveries'] > 50], dfecon[['Bowler','Economy']], on = 'Bowler')


## Bowler variance
df_bowler_summ['Runs_sq'] = 0
df_bowler_summ['Matches_played'] = 0
for i in range(len(df_bowler_summ)):
    for j in range(len(df_bowl_match)):
        if df_bowler_summ.iloc[i]['Bowler'] == df_bowl_match.iloc[j]['Bowler']:
            df_bowler_summ['Runs_sq'].iloc[i] += (df_bowl_match.iloc[j]['Batsman_runs'] +  df_bowl_match.iloc[j]['Extra_runs'])**2
            df_bowler_summ['Matches_played'].iloc[i] += 1

AvgRuns = df_bowler_summ['AvgBatsman_runs']+df_bowler_summ['AvgExtra_runs']
NumberMatches = df_bowler_summ['Matches_played']

df_bowler_summ['Var'] = 0

for i in range(len(df_bowler_summ)):
    if NumberMatches.loc[i] > 0:
        df_bowler_summ["Var"].loc[i] = (df_bowler_summ['Runs_sq'].loc[i] - NumberMatches.loc[i]*(AvgRuns.loc[i]**2))/(NumberMatches.loc[i] - 1)

 

df_bowler = pd.merge(df_bowler_summ, df[['Bowler', 'Bowling_team']].drop_duplicates(keep = 'first'), on = 'Bowler')
df_bowler.dropna(inplace=True)


In [387]:
df_bowler_summ

,Bowler,Batsman_runs,Extra_runs,wicket,Six,Four,Caught,Bowled,Runout,Caught_bowled,...,AvgBatsman_runs,AvgExtra_runs,AvgWicket,AvgSix,AvgFour,AvgDeliveries_bowled,Economy,Runs_sq,Matches_played,Var
0,Aamer Yamin,82,4,5,3,6,2,0,0,1,...,27.333333,1.333333,1.666667,1.000000,2.000000,24.666667,7.166667,2666,3,100.333333
1,Anwar Ali,272,10,6,15,21,5,0,0,0,...,30.222222,1.111111,0.666667,1.666667,2.333333,20.888889,9.400000,10144,9,163.500000
2,Bilawal Bhatti,93,13,1,2,9,1,0,0,0,...,18.600000,2.600000,0.200000,0.400000,1.800000,19.600000,6.625000,2500,5,63.200000
3,CJ Jordan,124,12,7,2,11,7,0,0,0,...,20.666667,2.000000,1.166667,0.333333,1.833333,20.000000,6.800000,3462,6,75.866667
4,DJG Sammy,96,9,2,4,10,2,0,0,0,...,16.000000,1.500000,0.333333,0.666667,1.666667,10.500000,10.500000,2167,6,65.900000
5,GD Elliott,112,3,5,5,9,2,0,0,0,...,16.000000,0.428571,0.714286,0.714286,1.285714,10.571429,9.583333,2139,7,41.619048
6,Hasan Ali,253,5,13,6,26,5,0,0,2,...,25.300000,0.500000,1.300000,0.600000,2.600000,22.100000,6.972973,7212,10,61.733333
7,Hasan Khan,213,19,9,9,14,3,0,0,1,...,21.300000,1.900000,0.900000,0.900000,1.400000,20.300000,7.030303,6486,10,122.622222
8,Imad Wasim,189,5,8,9,13,1,0,0,1,...,18.900000,0.500000,0.800000,0.900000,1.300000,17.600000,6.689655,3990,10,25.155556
9,Junaid Khan,96,5,3,7,9,2,0,0,0,...,48.000000,2.500000,1.500000,3.500000,4.500000,26.500000,12.625000,5105,2,4.500000


In [253]:
quadrant_chart(
    x=df_bowler['Economy'],
    y=df_bowler['wicket'],
    team=df_bowler['Bowling_team'],
    xtick_labels=['Low', 'High'],
    ytick_labels=['Low', 'High'],
    highlight_quadrants=[1]
)

In [260]:
#bowler box

trace0 = go.Box(y = df_bowler.loc[df_bowler['Bowling_team'] == 'Peshawar Zalmi', 'Economy'], name = 'Peshawar Zalmi', boxmean = 'sd', boxpoints = 'all')
trace1 = go.Box(y = df_bowler.loc[df_bowler['Bowling_team'] == 'Islamabad United', 'Economy'], name = 'Islamabad United', boxmean = 'sd', boxpoints = 'all')
trace2 = go.Box(y = df_bowler.loc[df_bowler['Bowling_team'] == 'Quetta Gladiators', 'Economy'], name = 'Quetta Gladiators', boxmean = 'sd', boxpoints = 'all')
trace3 = go.Box(y = df_bowler.loc[df_bowler['Bowling_team'] == 'Karachi Kings', 'Economy'], name = 'Karachi Kings' , boxmean = 'sd', boxpoints = 'all')
trace4 = go.Box(y = df_bowler.loc[df_bowler['Bowling_team'] == 'Lahore Qalandars', 'Economy'], name = 'Lahore Qalandars' , boxmean = 'sd', boxpoints = 'all')

data = [trace0, trace1, trace2, trace3, trace4]

fig111 = go.Figure(data = data)
fig111.layout.title = dict(text = "Bowler Economy distribution for each team" )
fig111.show()

##PESHAWAR ZALMI standout - 9 regular bowlwers, smallest median, 6 bowlers qith economy bteeer tahn 7
##Lahore comes close with 3 bowlers alomst less than 7

In [ ]:
# ##Setting up a ranking datafrane

# dfscore = df_batAvg[['Batsman_strike', 'Batting_team','AvgRuns_match', 'Strike_rate', 'Var']].rename(columns = {'Batsman_strike': 'Player', 'Batting_team':'Team','Var': 'Bat_var'}).merge(df_bowler[['Bowler', 'Economy', 'AvgWicket','AvgDeliveries_bowled', 'Var', 'Bowling_team']].rename(columns = {'Bowler':'Player','Var':'Bowl_var','Bowling_team':'Team'}), on = ['Player','Team'], how = 'outer').fillna(value = 0)                                                                                                          


##Player Ranking Metrics

# f = 'Strike_rate'
# g = 'AvgRuns_match'
# h = 'AvgWicket'


# Batsman = np.quantile(dfscore['AvgRuns_match'], 0.5)
# Batvarmean = dfscore.loc[dfscore['AvgRuns_match'] >= Batsman, "Bat_var"].mean()


# Bowlvarmean = dfscore.loc[dfscore['AvgDeliveries_bowled'] >= 0, "Bowl_var"].mean()

# dfscore['BatVarRank'] = np.zeros(len(dfscore))

# dfscore['BowlVarRank'] = np.zeros(len(dfscore))


## foour quarters 0,1,2,3
## should be based on np quartiles
## lets make it if score is 0 then 0, less than 0.25:1 less than 0.5:2, 0.75:3 greater than 0.75:4

##Awarding Average Runs
# def greaterf(x):
#         if x == 0:
#             return 0
#         if x >= 1.25 * dfscore[f].mean():
#             return 3
#         if x<= 0.75 *dfscore[f].mean():
#             return  1
#         else:
#             return 2


# def greaterg(x):
#         if x == 0:
#             return 0
#         if x >= 1.25 * dfscore[g].mean():
#             return 3
#         if x<= 0.75 *dfscore[g].mean():
#             return  1
#         else:
#             return 2

# def greaterh(x):
#         if x == 0:
#             return 0
#         if x >= 1.25 * dfscore[h].mean():
#             return 3
#         if x<= 0.75 *dfscore[h].mean():
#             return  1
#         else:
#             return 2


# dfscore[f + str('Rank')] = dfscore[f].map(lambda x: greaterf(x))    
# dfscore[g + str('Rank')] = dfscore[g].map(lambda x: greaterg(x))
# dfscore[h + str('Rank')] = dfscore[h].map(lambda x: greaterh(x))

# def ecorank(x):
#     if x == 0:
#         return 0
#     elif x >= 1.25 * dfscore.loc[dfscore['AvgDeliveries_bowled'] >= 0, "Economy"].mean():
#         return 1
#     elif x <= 0.75 * dfscore.loc[dfscore['AvgDeliveries_bowled'] >= 0, "Economy"].mean():
#         return 3
#     else:
#         return 2
    
# dfscore['EconomyRank'] = dfscore['Economy'].map(lambda x: ecorank(x))


# for i in range(len(dfscore)):
#     if dfscore.iloc[i]['AvgRuns_match'] <= Batsman:
#         dfscore['BatVarRank'].iloc[i] = 0
#     elif dfscore['Bat_var'].iloc[i] >= (1.25 * Batvarmean):  
#         dfscore['BatVarRank'].iloc[i] = 1
#     elif dfscore.iloc[i]['Bat_var'] <= 0.75 * Bowlvarmean:
#         dfscore['BatVarRank'].iloc[i] = 3
#     else:
#         dfscore['BatVarRank'].iloc[i] = 2
        
        
        
# for i in range(len(dfscore)):
#     if dfscore.iloc[i]['AvgDeliveries_bowled'] == 0:
#         dfscore['BowlVarRank'].iloc[i] = 0
#     elif dfscore['Bowl_var'].iloc[i] >= (1.25 * Bowlvarmean):  
#         dfscore['BowlVarRank'].iloc[i] = 1
#     elif dfscore.iloc[i]['Bowl_var'] <= 0.75 * Bowlvarmean:
#         dfscore['BowlVarRank'].iloc[i] = 3
#     else:
#         dfscore['BowlVarRank'].iloc[i] = 2



# ##Creating PlayerScore column as the sum of all metrics

# dfscore['PlayerScore'] = dfscore['AvgRuns_matchRank'] + dfscore['Strike_rateRank'] + dfscore['AvgWicketRank'] + dfscore['BatVarRank'] + dfscore['BowlVarRank'] + dfscore['EconomyRank']


# ##Ranking levels

# def mvprank(x):
#     if x >= np.quantile(dfscore['PlayerScore'], 0.90):
#         return 4
#     elif x >= np.quantile(dfscore['PlayerScore'], 0.75):
#             return 3
#     elif x <= np.quantile(dfscore['PlayerScore'], 0.50):
#             return 1
#     else:
#         return 2
    
# dfscore['PlayerRank'] = dfscore['PlayerScore'].map(lambda x: mvprank(x))
# dfscore['MVP'] = np.zeros(len(dfscore))
# dfscore.loc[dfscore['PlayerRank']==4,'MVP'] = 'MVP'
# dfscore.loc[dfscore['PlayerRank']==3, 'MVP'] = 'VP'
# dfscore.loc[dfscore['PlayerRank'] <3, 'MVP'] = 'Below Average'


# ## Top 10 Players
# dfscore.sort_values(by = 'PlayerScore', ascending = False).head(10)


# #Boxplot


# trace0 = go.Box(y = dfscore.loc[dfscore['Team'] == 'Peshawar Zalmi', 'PlayerScore'], name = 'Peshawar Zalmi', boxmean = 'sd', boxpoints = 'all')
# trace1 = go.Box(y = dfscore.loc[dfscore['Team'] == 'Islamabad United', 'PlayerScore'], name = 'Islamabad United', boxmean = 'sd', boxpoints = 'all')
# trace2 = go.Box(y = dfscore.loc[dfscore['Team'] == 'Quetta Gladiators', 'PlayerScore'], name = 'Quetta Gladiators', boxmean = 'sd', boxpoints = 'all')
# trace3 = go.Box(y = dfscore.loc[dfscore['Team'] == 'Karachi Kings', 'PlayerScore'], name = 'Karachi Kings' , boxmean = 'sd', boxpoints = 'all')
# trace4 = go.Box(y = dfscore.loc[dfscore['Team'] == 'Lahore Qalandars', 'PlayerScore'], name = 'Lahore Qalandars' , boxmean = 'sd', boxpoints = 'all')

# data = [trace0, trace1, trace2, trace3, trace4]

# fig111 = go.Figure(data = data)
# fig111.layout.title = dict(text = "Player Score distribution for each team" )

# plotly.offline.plot(fig111, include_plotlyjs = False, output_type = 'div')

#histogram


In [405]:
dfscore = df_batAvg[['Batsman_strike', 'Batting_team','TotalRuns','AvgRuns_match', 'Strike_rate', 'Var']].rename(columns = {'Batsman_strike': 'Player', 'Batting_team':'Team','Var': 'Bat_var'}).merge(df_bowler[['Bowler', 'Economy','wicket' ,'AvgWicket','AvgDeliveries_bowled', 'TotalDeliveries', 'Var', 'Bowling_team']].rename(columns = {'Bowler':'Player','Var':'Bowl_var','Bowling_team':'Team'}), on = ['Player','Team'], how = 'outer').fillna(value = 0)     

# h = 'AvgWicket'
metricsPostiveIncrement = ['Strike_rate', 'TotalRuns', 'wicket']
metricsNegativeIncrement = ['Economy', 'Bat_var', 'Bowl_var']

def awardingPositiveIncrement(i,x):
        if x == 0:
            return 0
        if x <= np.quantile(dfscore[i],0.25):
            return 1
        if x<=  np.quantile(dfscore[i],0.5):
            return  2
        if x<=  np.quantile(dfscore[i],0.75):
            return  3
        else:
            return 4

def awardingNegativeIncrement(i,x):
    if x == 0:
        return 0
    if x >=  1.5*dfscore[i].mean():
        return 1
    if ((x >=  dfscore[i].mean()) & (x < 1.5*dfscore[i].mean())):
        return 2
    if x <= 0.25*dfscore[i].mean():
        return 4
    else:
        return  3

def awardingMetric():
    for i in metricsPostiveIncrement:
            dfscore[i+'Rank'] = dfscore[i].map(lambda x: awardingPositiveIncrement(i, x)) 
    for i in metricsNegativeIncrement:
            dfscore[i+'Rank'] = dfscore[i].map(lambda x: awardingNegativeIncrement(i , x)) 
    return dfscore 



##Awarding Average Runs
# def greaterg(x):
#         if x == 0:
#             return 0
#         if x =< 1.25 * dfscore[g].mean():
#             return 3
#         if x<= 0.75 *dfscore[g].mean():
#             return  1
#         else:
#             return 2

##Awarding Wickets
# def greaterh(x):
#       for i in Metric:
#         if x == 0:
#             return 0
#         if x >= 1.25 * dfscore[h].mean():
#             return 3
#         if x<= 0.75 *dfscore[h].mean():
#             return  1
#         else:
#             return 2


#
##awarding players
# dfscore[Strike_rate + str('Rank')] = dfscore[Strike_rate].map(lambda x: awardingMetric(x))    
# dfscore[TotalRuns + str('Rank')] = dfscore[TotalRuns].map(lambda x: awardingMetric(x))
# dfscore[h + str('Rank')] = dfscore[h].map(lambda x: greaterh(x))




In [406]:
awardingMetric()

,Player,Team,TotalRuns,AvgRuns_match,Strike_rate,Bat_var,Economy,wicket,AvgWicket,AvgDeliveries_bowled,TotalDeliveries,Bowl_var,Strike_rateRank,TotalRunsRank,wicketRank,EconomyRank,Bat_varRank,Bowl_varRank
0,Kamran Akmal,Peshawar Zalmi,353.0,32.090909,128.832117,1343.890909,0.000000,0.0,0.000000,0.000000,0.0,0.000000,4,4,0,0,1,0
1,Babar Azam,Karachi Kings,291.0,29.100000,109.811321,387.877778,0.000000,0.0,0.000000,0.000000,0.0,0.000000,3,4,0,0,1,0
2,DR Smith,Islamabad United,274.0,30.444444,110.931174,579.277778,0.000000,0.0,0.000000,0.000000,0.0,0.000000,3,4,0,0,1,0
3,RR Rossouw,Quetta Gladiators,255.0,31.875000,121.428571,632.125000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,3,4,0,0,1,0
4,KP Pietersen,Quetta Gladiators,241.0,34.428571,148.765432,1249.619048,0.000000,0.0,0.000000,0.000000,0.0,0.000000,4,4,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,Usman Khan,Karachi Kings,0.0,0.000000,0.000000,0.000000,7.653846,9.0,1.125000,20.250000,162.0,82.982143,0,0,4,1,0,1
78,Rumman Raees,Islamabad United,0.0,0.000000,0.000000,0.000000,5.857143,13.0,1.857143,23.000000,161.0,16.285714,0,0,4,1,0,3
79,Mohammad Irfan,Islamabad United,0.0,0.000000,0.000000,0.000000,7.533333,8.0,1.000000,22.875000,183.0,73.071429,0,0,4,1,0,1
80,Mohammad Asghar,Peshawar Zalmi,0.0,0.000000,0.000000,0.000000,8.764706,8.0,1.142857,14.714286,103.0,89.238095,0,0,4,1,0,1


In [410]:
##Creating PlayerScore column as the sum of all metrics

dfscore['PlayerScore'] = dfscore['TotalRunsRank'] + dfscore['Strike_rateRank'] + dfscore['wicketRank'] + dfscore['EconomyRank'] + dfscore['Bat_varRank'] + dfscore['Bowl_varRank']

##Histogram with distribution

team_list = list(dfscore['Team'].unique())

trace0 = go.Histogram(x= dfscore.loc[dfscore['Team'] == team_list[0], 'PlayerScore'], opacity = 0.75, name = team_list[0])
trace1 = go.Histogram(x= dfscore.loc[dfscore['Team'] == team_list[1], 'PlayerScore'], opacity = 0.75, name = team_list[1])
trace2 = go.Histogram(x= dfscore.loc[dfscore['Team'] == team_list[2], 'PlayerScore'], opacity = 0.75, name = team_list[2])
trace3 = go.Histogram(x= dfscore.loc[dfscore['Team'] == team_list[3], 'PlayerScore'], opacity = 0.75, name = team_list[3])
trace4 = go.Histogram(x= dfscore.loc[dfscore['Team'] == team_list[4], 'PlayerScore'], opacity = 0.75, name = team_list[4])

fig4 = make_subplots(rows =3, cols = 2)
fig4.append_trace(trace0,1,1)
fig4.append_trace(trace1,2,1)
fig4.append_trace(trace2,2,2)
fig4.append_trace(trace3,3,1)
fig4.append_trace(trace4,3,2)

fig4.layout.title = dict(text = 'Distribution of average batsman scores')
fig4.layout.xaxis = dict(tickmode = 'linear', dtick = 10)

fig4.show()

##Peshawar and quetta have the clear standout players
##Makes sensse as they both were the tournamnet finalist with Peshawar winning the final
